### 作業目的:了解Ensemble中的Blending與Stacking

### Question: 請描述Blending與Stacking的差異

## Answer:
### Stacking
 使用 kfoldCV來進行第二階段模型的特徵抽取，而且是只針對每一次保留的val_set去進行預測proba留存，並將之與模型的ground_truth一起訓練二階段模型

### blending 
只需要一開始設定好預留的validation set(holdout set)即可完成一樣的事情，計算量更少更簡單，分享更簡便，但可能會對holdout set overfitting是需要注意的地方

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [1]:
def Stacking(model,train,y,test,n_fold):
  folds=StratifiedKFold(n_splits=n_fold,random_state=1)
  test_pred=np.empty((test.shape[0],1),float)
  train_pred=np.empty((0,1),float)
  
  for train_indices,val_indices in folds.split(train,y.values):
    x_train,x_val=train.iloc[train_indices],train.iloc[val_indices]
    y_train,y_val=y.iloc[train_indices],y.iloc[val_indices]
    model.fit(X=x_train,y=y_train)
    train_pred=np.append(train_pred,model.predict(x_val))
    test_pred=np.append(test_pred,model.predict(test))
    
  return test_pred.reshape(-1,1),train_pred

In [ ]:
# initialise the base models
model_1 = tree.DecisionTreeClassifier(random_state=1)
model_2 = KNeighborsClassifier()

# train base models and create new featurs
test_pred_1 ,train_pred_1=Stacking(model=model_1,n_fold=10, train=x_train,test=x_test,y=y_train)
test_pred_2 ,train_pred_2=Stacking(model=model_2,n_fold=10,train=x_train,test=x_test,y=y_train)

# convert into dataframe for later use
train_pred_1=pd.DataFrame(train_pred_1)
test_pred_1=pd.DataFrame(test_pred_1)
train_pred_2=pd.DataFrame(train_pred_2)
test_pred_2=pd.DataFrame(test_pred_2)

In [ ]:
df = pd.concat([train_pred_1, train_pred_2], axis=1)
df_test = pd.concat([test_pred_1, test_pred_2], axis=1)

model = LogisticRegression(random_state=1)
model.fit(df,y_train)
model.score(df_test, y_test)

In [ ]:
model_1 = tree.DecisionTreeClassifier()
model_1.fit(x_train, y_train)
holdout_pred_1=model_1.predict(x_val)
test_pred_1=model_1.predict(x_test)
holdout_pred_1=pd.DataFrame(holdout_pred_1)
test_pred1=pd.DataFrame(test_pred1)

model_2 = KNeighborsClassifier()
model_2.fit(x_train,y_train)
holdout_pred_2=model_2.predict(x_val)
test_pred_2=model_2.predict(x_test)
holdout_pred_2=pd.DataFrame(holdout_pred_2)
test_pred_2=pd.DataFrame(test_pred_2)

In [ ]:
df_holdout=pd.concat([x_val, holdout_pred_1,holdout_pred_2],axis=1)
df_test=pd.concat([x_test, test_pred_1,test_pred_2],axis=1)

model = LogisticRegression()
model.fit(df_holdout,y_holdout)
model.score(df_test,y_test)